## import

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import pandas as pd

/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <BE0CCD9A-269A-30E2-A23C-DA45E89EBB1F> /Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/lib/python3.10/lib-dynload/../../libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/bin/../lib/libjpeg.8.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS@rpath/libjpeg.8.dylib' (no such file), '/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/torchvision/../../../

In [61]:
import os
import shutil

# 데이터셋 경로 설정하기
dataset_path = './usg1_dataset/train_images/'
dir = "./usg1_dataset/"
image_dir = dir+'train_images/'
train_csv_dir = dir+'train.csv'
train_copy_dir = './usg1_dataset/train_copy/'
test_dir = './usg1_dataset/test_images/'
split_label_dir = dir + 'train_split_label/'

In [51]:
os.makedirs(split_label_dir, exist_ok=True)

train_dir = split_label_dir + '/train/'
valid_dir = split_label_dir + '/valid/'
test_dir = split_label_dir + '/test/'

os.makedirs(split_label_dir + '/train', exist_ok=True)
os.makedirs(split_label_dir + '/valid', exist_ok=True)
os.makedirs(split_label_dir + '/test', exist_ok=True)


shutil.rmtree(train_dir)
shutil.rmtree(valid_dir)
shutil.rmtree(test_dir)

os.makedirs(split_label_dir + '/train', exist_ok=True)
os.makedirs(split_label_dir + '/valid', exist_ok=True)
os.makedirs(split_label_dir + '/test', exist_ok=True)


# 레이블 이름 설정하기
train_labels = pd.read_csv(train_csv_dir)
labels = train_labels['label'].to_list() # 레이블 리스트
labels_str = []
for label in labels:
    labels_str.append(str(label))

# 하위 폴더 생성하기
for label in labels_str:
    os.makedirs(train_dir + label, exist_ok=True)
    os.makedirs(valid_dir + label, exist_ok=True)

In [57]:
import os

# 파일 이름 변경하기 ""

# 1) copy : train_images => train_copy (and test)

# train

#os.makedirs(train_copy_dir)
shutil.rmtree(train_copy_dir)
os.makedirs(train_copy_dir)

filenames = natsorted(os.listdir(image_dir))
for filename in filenames:
    if filename.endswith('.jpg'):  # 변경할 파일 확장자 지정

        name = os.path.join(image_dir, filename)
        shutil.copy(name, train_copy_dir)


# test

split_test_dir = split_label_dir + 'test/'

test_filenames = natsorted(os.listdir(test_dir))

for filename in test_filenames:
    if filename.endswith('.jpg'):  # 변경할 파일 확장자 지정

        name = os.path.join(test_dir, filename)
        shutil.copy(name, split_test_dir)

In [63]:
# test

split_test_dir = split_label_dir + 'test/'
#os.makedirs(test_copy_dir)
shutil.rmtree(test_copy_dir)
os.makedirs(test_copy_dir)

test_filenames = natsorted(os.listdir(test_dir))

for filename in test_filenames:
    if filename.endswith('.jpg'):  # 변경할 파일 확장자 지정

        name = os.path.join(test_dir, filename)
        shutil.copy(name, split_test_dir)

In [48]:
# 2) rename

from natsort import natsorted

filenames = natsorted(os.listdir(train_copy_dir))
for idx, filename in enumerate(filenames):
        label = labels_str[idx]

        name = os.path.join(train_copy_dir, filename)
        labeled_name = os.path.join(train_copy_dir, os.path.splitext(filename)[0]+ '_' +label+'.jpg')
        os.rename(name, labeled_name)

In [22]:
# 2) split 
from sklearn.model_selection import train_test_split

filenames = natsorted(os.listdir(train_copy_dir))

train_filenames, valid_filenames = train_test_split(filenames, test_size=0.1, random_state=42)

train_filenames = natsorted(train_filenames)
valid_filenames = natsorted(valid_filenames)

print('train\n',train_filenames[:5])
print('valid\n',valid_filenames[:5])

train
 ['000000_1.jpg', '000001_8.jpg', '000002_0.jpg', '000003_1.jpg', '000004_8.jpg']
valid
 ['000008_9.jpg', '000014_1.jpg', '000019_0.jpg', '000031_0.jpg', '000033_6.jpg']


In [23]:
# 이미지 파일 이동하기

# train
for file in train_filenames:
    if file.endswith('.jpg'):
        filename = os.path.splitext(file)[0]
        label = filename.split('_')[1]
        shutil.copy(train_copy_dir+ '/' +file, train_dir + label + '/' + file)

# valid
for file in valid_filenames:
    if file.endswith('.jpg'):
        filename = os.path.splitext(file)[0]
        label = filename.split('_')[1]
        shutil.copy(train_copy_dir+ '/' +file, valid_dir + label + '/' + file)

In [24]:
# 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir,
                                      transform=transforms.ToTensor())

valid_dataset = datasets.ImageFolder(root=valid_dir,
                                      transform=transforms.ToTensor())

In [25]:
# 데이터로더 생성하기
batch_size = 8
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)